In [1]:
import os
import re
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from botorch import fit_gpytorch_mll
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.likelihoods import FixedNoiseGaussianLikelihood
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood

from gskgpr import GaussianStringKernelGP
from seq2ascii import Seq2Ascii

In [2]:
REF_POINT = torch.Tensor([-50, -50])
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
def load_json_res(pcc, data_dir):
    with open(f"{data_dir}/{pcc}_FEN.JSON") as f:
        rep = json.load(f)
    F_fen = rep["FE"]
    F_fen_err = rep["FE_error"]

    with open(f"{data_dir}/{pcc}_DEC.JSON") as f:
        rep = json.load(f)
    F_dec = rep["FE"]
    F_dec_err = rep["FE_error"]
    return {"PCC": [rep["PCC"]], "F_FEN": [float(F_fen)], "err_FEN": [float(F_fen_err)],
             "F_DEC": [float(F_dec)], "err_DEC": [float(F_dec_err)]}

def load_data(data_dir):
    PCC_list = []
    for folder in os.listdir(data_dir):
        if re.match("[A-Z]{5}_[A-Z]{3}", folder):
            PCC_list.append(folder.split("_")[0])

    PCC_list = set(PCC_list)
    data = []
    for pcc in PCC_list:
        try:
            data.append(pd.DataFrame(load_json_res(pcc, data_dir)))
        except:
            print(f"Skipping {pcc}.")

    data = pd.concat(data)
    data.reset_index(inplace=True, drop=True)
    return data

def initialize_model(train_x, train_y, err_y, translator):
    models = [
        GaussianStringKernelGP(train_x=train_x, train_y=train_y[:, 0], 
                            likelihood=FixedNoiseGaussianLikelihood(noise=err_y[:, 0]), 
                            translator=translator),
        GaussianStringKernelGP(train_x=train_x, train_y=train_y[:, 1],
                            likelihood=FixedNoiseGaussianLikelihood(noise=err_y[:, 1]), 
                            translator=translator)
    ]
    model = ModelListGP(*models).to(device)
    mll = SumMarginalLogLikelihood(model.likelihood, model).to(device)
    return model, mll

def fit_gpytorch_model(mll, optimizer, n_iters=100):
    for i in range(n_iters):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = mll.model(*mll.model.train_inputs)
        # Calc loss and backprop gradients
        loss = -mll(output, mll.model.train_targets)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f   sigma11: %.3f   sigma21: %.3f   sigma12: %.3f   sigma22: %.3f' % (
            i + 1, n_iters, loss.item(),
            mll.model.models[0].covar_module.sigma1.item(),
            mll.model.models[0].covar_module.sigma2.item(),
            mll.model.models[1].covar_module.sigma1.item(),
            mll.model.models[1].covar_module.sigma2.item(),
        ))
        optimizer.step()

In [4]:
dataset = load_data("/project2/andrewferguson/armin/FE_DATA/results/")
dataset["ddG_sen"] = -1*dataset.F_FEN
dataset["ddG_spe"] = dataset.F_DEC-dataset.F_FEN
dataset["sen_var"] = dataset.err_FEN
dataset["spe_var"] = np.sqrt(dataset.err_FEN**2 + dataset.err_DEC**2)
dataset.ddG_sen = (dataset.ddG_sen - dataset.ddG_sen.mean())/dataset.ddG_sen.std()
dataset.sen_var = dataset.sen_var/dataset.ddG_sen.std()
dataset.ddG_spe = (dataset.ddG_spe - dataset.ddG_spe.mean())/dataset.ddG_spe.std()
dataset.spe_var = dataset.spe_var/dataset.ddG_spe.std()

In [5]:
translator = Seq2Ascii("/project/andrewferguson/armin/HTVS_Fentanyl/MFMOBO/AA.blosum62.pckl")

fspace = []
with open("/project/andrewferguson/armin/HTVS_Fentanyl/gen_input_space/full_space.txt") as f:
    line = f.readline()
    while line:
        fspace.append(line.split()[0])
        line = f.readline()

translator.fit(fspace)

In [6]:
encoded_x = translator.encode_to_int(dataset.PCC.to_list()).to(device)
FE_sen = torch.tensor(dataset.ddG_sen.to_numpy()).float().to(device)
FE_sen_var = torch.tensor(dataset.sen_var.to_numpy()).float().to(device)
FE_spe = torch.tensor(dataset.ddG_spe.to_numpy()).float().to(device)
FE_spe_var = torch.tensor(dataset.spe_var.to_numpy()).float().to(device)
train_y = torch.cat([FE_sen.view(-1, 1), FE_spe.view(-1, 1)], dim=1)
err_y = torch.cat([FE_sen_var.view(-1, 1), FE_spe_var.view(-1, 1)], dim=1)

In [7]:
model, mll = initialize_model(encoded_x, train_y, err_y, translator)

In [8]:
dataset

,PCC,F_FEN,err_FEN,F_DEC,err_DEC,ddG_sen,ddG_spe,sen_var,spe_var
0,GHGGF,-7.458785,0.325139,-5.914494,0.117817,-0.309527,-0.294839,0.325139,0.345827
1,GTGSG,-6.978182,0.306770,-6.713622,0.434684,-0.439369,-0.731104,0.306770,0.532032
2,TERPF,-5.519027,0.114119,-4.606593,0.060435,-0.833582,-0.510241,0.114119,0.129133
3,WWWWW,-14.399965,0.466565,-6.271820,0.166044,1.565739,1.949624,0.466565,0.495231
4,GGDEK,-7.550094,0.305804,-5.107204,0.178314,-0.284858,0.011497,0.305804,0.353995
5,YADAL,-9.317697,0.777650,-7.286629,0.428413,0.192687,-0.128894,0.777650,0.887850
6,HHHHH,-11.312336,0.459934,-9.105205,0.363219,0.731569,-0.068874,0.459934,0.586061
7,AWPVN,-9.215318,0.437208,-7.464716,0.241339,0.165027,-0.224507,0.437208,0.499395
8,GRGFG,-7.602260,0.106252,-7.410885,0.638032,-0.270765,-0.756053,0.106252,0.646818
9,PRHTS,-10.580213,0.359631,-7.314180,0.245566,0.533775,0.292110,0.359631,0.435474


In [9]:
del fspace

In [10]:
fit_gpytorch_mll(mll)

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0-1): 2 x FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0-1): 2 x GaussianStringKernelGP(
        (likelihood): FixedNoiseGaussianLikelihood(
          (noise_covar): FixedGaussianNoise()
        )
        (mean_module): ConstantMean()
        (covar_module): GenericStringKernel(
          (raw_sigma1_constraint): Positive()
          (raw_sigma2_constraint): Positive()
        )
      )
    )
    (likelihood): LikelihoodList(
      (likelihoods): ModuleList(
        (0-1): 2 x FixedNoiseGaussianLikelihood(
          (noise_covar): FixedGaussianNoise()
        )
      )
    )
  )
  (mlls): ModuleList(
    (0-1): 2 x ExactMarginalLogLikelihood(
      (likelihood): FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
      (model): GaussianStringKernelGP(
        

In [11]:
full_space = torch.as_tensor(list(translator.int2str.keys())).view(-1, 1).to(device)

In [ ]:
post = model.posterior(full_space[::10])

In [ ]:
#plt.errorbar(x = post.mean[:, 0].detach().numpy(), y = post.mean[:, 1].detach().numpy(), xerr=post.variance[:, 0].sqrt().detach().numpy(), yerr=post.variance[:, 1].sqrt().detach().numpy(), fmt='o')
plt.scatter(x = post.mean[:, 0].detach().numpy(), y = post.mean[:, 1].detach().numpy(), color='blue', label="Predicted")
plt.scatter(x = train_y[:, 0].detach().numpy(), y = train_y[:, 1].detach().numpy(), color='red', label="Training")
new_x = ["WWWWL", "WWWWY", "WWYHV"]
new_x = translator.encode_to_int(new_x).to(device)
plt.scatter(x = post.mean[new_x, 0].detach().numpy(), y = post.mean[new_x, 1].detach().numpy(), color='green', label="New Candidates")
plt.xlabel(r"-$\Delta G_{FEN}$")
plt.ylabel(r"$\Delta G_{DEC}-\Delta G_{FEN}$")
plt.legend()
plt.show()

In [ ]:
post_train = model.posterior(encoded_x)

In [ ]:
plt.errorbar(x = post_train.mean[:, 0].detach().numpy(), y = post_train.mean[:, 1].detach().numpy(),
             xerr=post_train.variance[:, 0].sqrt().detach().numpy(), yerr=post_train.variance[:, 1].sqrt().detach().numpy(), fmt='o', label="Predicted")
plt.errorbar(x = train_y[:, 0].detach().numpy(), y = train_y[:, 1].detach().numpy(),
             xerr=err_y[:, 0].sqrt().detach().numpy(), yerr=err_y[:, 1].sqrt().detach().numpy(), fmt='o', label="Training")
plt.xlabel(r"-$\Delta G_{FEN}$")
plt.ylabel(r"$\Delta G_{DEC}-\Delta G_{FEN}$")
plt.legend()
plt.show()

In [ ]:
plt.bar(dataset.PCC.to_list(), post_train.mean[:, 0].detach().numpy()-train_y[:, 0].detach().numpy(),
        yerr=post_train.variance[:, 0].sqrt().detach().numpy(), label="Senstivity")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.bar(dataset.PCC.to_list(), post_train.mean[:, 1].detach().numpy()-train_y[:, 1].detach().numpy(),
        yerr=post_train.variance[:, 1].sqrt().detach().numpy(), label="Specificity")
plt.xticks(rotation=90)
plt.show()